# Capstone Project - Week 5

## 1. Introduction

The 2019 Thai general election was held in Thailand on 24 March 2019. The elections selected the five hundred members of the new House of Representatives, the previous House having been dissolved by the coup. The 350 constituency seats are won by first-past-the-post voting as in previous elections. However, the 150 party list seats act as levelling seats, and are allocated so as to give each party a total number of seats proportional to the nationwide number of votes they received.

In 2018, the Election Commission (ECT) was tasked with drawing up new district boundaries. The move sparked outrage from the Pheu Thai and Democrat parties and the watchdog organisation Open Forum for Democracy Foundation. They argued the purpose of the delay was to enable the ECT to draw constituency boundaries that favoured PM Prayut's Palang Pracharath Party.

Therefore, the objective of this project is to investigate the boundary issue by studying correlations between neighborhoods in  Thailand and voters' behavior. In this project, we may measure voters' behavior using the most popular party for each constituency. To simplify the analysis, we will focus only on the 350 constituency seats. 

#### Target Audience
People who are interested in 2019 Election fever and wondered about the constituency boundary controversy

## 2. Data

### 2.1 Constituency Data
It was the first election since the 2014 Thai coup d'état that installed coup leader General Prayut Chan-o-cha as prime minister, and the first held in accordance with the 2017 constitution, which was drafted under the ruling military junta. Therefore, several news agencies had developed real-time election monitoring website. For example:

* https://elect.thematter.co/
* https://vote.workpointnews.com/
* https://web.facebook.com/thestandardth/photos/a.1725541161072102/2074459142846967/?type=3&theater
* https://vote62.com/

In this project, we will use JSON data file from https://github.com/codeforthailand/dataset-election-62-candidates/blob/master/data/election-zones.json which was used to produce constituency query service page (https://elect.in.th/candidates/) Below is the example of JSON file.

In [32]:
import requests

In [33]:
url_zone = 'https://raw.githubusercontent.com/pornvutp/Coursera_Capstone/master/election-zones.json'
result_zone = requests.get(url_zone).json()
result_zone

[{'areas': [{'area': 'ปลายพระยา',
    'exterior': [],
    'interior': [],
    'subinterior': []},
   {'area': 'อ่าวลึก', 'exterior': [], 'interior': [], 'subinterior': []},
   {'area': 'เขาพนม',
    'exterior': [],
    'interior': ['หน้าเขา', 'เขาดิน'],
    'subinterior': []},
   {'area': 'เมืองกระบี่', 'exterior': [], 'interior': [], 'subinterior': []}],
  'prefixes': {'area': 'อำเภอ', 'sub_area': 'ตำบล'},
  'province': 'กระบี่',
  'zone': 1},
 {'areas': [{'area': 'คลองท่อม',
    'exterior': [],
    'interior': [],
    'subinterior': []},
   {'area': 'ลำทับ', 'exterior': [], 'interior': [], 'subinterior': []},
   {'area': 'เกาะลันตา', 'exterior': [], 'interior': [], 'subinterior': []},
   {'area': 'เขาพนม',
    'exterior': ['หน้าเขา', 'เขาดิน'],
    'interior': [],
    'subinterior': []},
   {'area': 'เหนือคลอง', 'exterior': [], 'interior': [], 'subinterior': []}],
  'prefixes': {'area': 'อำเภอ', 'sub_area': 'ตำบล'},
  'province': 'กระบี่',
  'zone': 2},
 {'areas': [{'area': 'ดุสิต',


#### Geospatial Data (Latitude/Longitude)

For each constituency (zone), area (AMPHOE) and province (CHANGWAT) parameters will be used as a key for extracting latitude and longitude from Thailand Open Government Data (<a href="https://www.data.go.th/DatasetDetail.aspx?id=c6d42e1b-3219-47e1-b6b7-dfe914f27910">Link</a>).

<img src="https://raw.githubusercontent.com/pornvutp/Coursera_Capstone/master/lat-long.png" style="width:75%">

### 2.2 Neighborhood Data

We apply Foursquare API to explore the nearby venues according to the latitude/longitude data of each constituency. Then, venue categories are used as a main feature to cluster neighborhoods in Thailand (similar analysis as Manhattan and Toronto examples).

### 2.3 Election Result
We use the election result from The Election Commission of Thailand (ECT) official website (<a href="https://www.ect.go.th/ewt/ewt/ect_th/download/article/article_20190328165029.pdf">Link</a>). The election result is sorted by constituency and popularity of candidate as the picture below (translation in red color). However, data preparation is needed since the file is in PDF format.

<img src="https://raw.githubusercontent.com/pornvutp/Coursera_Capstone/master/ResultTable.png" style="width:75%px">


After converting PDF into XLSX format using Adobe Acrobat Pro with some Microsoft Excel functions, the file will be like this below.

<img src="https://raw.githubusercontent.com/pornvutp/Coursera_Capstone/master/result_xlsx.png" style="width:40%px">


For further analysis, this result may be modelled using classification algorithm since there are a few parties holding majority from 350 constituencies.

<img src="https://raw.githubusercontent.com/pornvutp/Coursera_Capstone/master/350.png" style="width:75%px">


## 3. Methodologies

### 3.1 Import Libraries

In [34]:
from bs4 import BeautifulSoup

import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!pip install folium
#!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

import statistics

print('Libraries imported.')

tensorflow 1.3.0 requires tensorflow-tensorboard<0.2.0,>=0.1.0, which is not installed.
Libraries imported.


### 3.2 Data Preprocessing

#### 3.2.1 Geospatial Data (Latitude/Longitude)

In [35]:
# Import XLSX data and convert into dataframe format

df_coord = pd.read_excel('https://github.com/pornvutp/Coursera_Capstone/blob/master/area_coord.xlsx?raw=true')
df_coord.head()

,AD_LEVEL,TA_ID,TAMBON_T,TAMBON_E,AM_ID,AMPHOE_T,AMPHOE_E,CH_ID,CHANGWAT_T,CHANGWAT_E,LAT,LONG
0,4,910106,ต. เกาะสาหร่าย,Ko Sarai,9101,อ. เมืองสตูล,Mueang Satun,91,จ. สตูล,Satun,6.546,99.706
1,4,210114,ต. มาบตาพุด,Maptaphut,2101,อ. เมืองระยอง,Mueang Rayong,21,จ. ระยอง,Rayong,12.646,101.171
2,4,210114,ต. มาบตาพุด,Maptaphut,2101,อ. เมืองระยอง,Mueang Rayong,21,จ. ระยอง,Rayong,12.645,101.170
3,4,210114,ต. มาบตาพุด,Maptaphut,2101,อ. เมืองระยอง,Mueang Rayong,21,จ. ระยอง,Rayong,12.649,101.174
4,4,210114,ต. มาบตาพุด,Maptaphut,2101,อ. เมืองระยอง,Mueang Rayong,21,จ. ระยอง,Rayong,12.644,101.169


In [36]:
# Select some useful columns

df_coord = df_coord[['CHANGWAT_T','CHANGWAT_E','AMPHOE_T','AMPHOE_E','LAT','LONG']]

df_coord = df_coord.replace(r'จ. ', '',regex=True)
df_coord = df_coord.replace(r'อ. ', '',regex=True)
df_coord = df_coord.replace(r'เขต ', '',regex=True)

df_coord.head()

,CHANGWAT_T,CHANGWAT_E,AMPHOE_T,AMPHOE_E,LAT,LONG
0,สตูล,Satun,เมืองสตูล,Mueang Satun,6.546,99.706
1,ระยอง,Rayong,เมืองระยอง,Mueang Rayong,12.646,101.171
2,ระยอง,Rayong,เมืองระยอง,Mueang Rayong,12.645,101.170
3,ระยอง,Rayong,เมืองระยอง,Mueang Rayong,12.649,101.174
4,ระยอง,Rayong,เมืองระยอง,Mueang Rayong,12.644,101.169


In [37]:
df_coord_dict = df_coord.copy()

df_coord_dict = df_coord_dict.drop_duplicates(subset="AMPHOE_E").reset_index()

df_coord_dict = df_coord_dict[['CHANGWAT_T','CHANGWAT_E','AMPHOE_T','AMPHOE_E']]

df_coord_dict.head(10)

,CHANGWAT_T,CHANGWAT_E,AMPHOE_T,AMPHOE_E
0,สตูล,Satun,เมืองสตูล,Mueang Satun
1,ระยอง,Rayong,เมืองระยอง,Mueang Rayong
2,ชุมพร,Chumphon,เมืองชุมพร,Mueang Chumphon
3,ชุมพร,Chumphon,สวี,Sawi
4,มหาสารคาม,Maha Sarakham,เมืองมหาสารคาม,Mueang Maha Sarakham
5,สุราษฎร์ธานี,Surat Thani,เกาะสมุย,Ko Samui
6,ชลบุรี,Chon Buri,สัตหีบ,Sattahip
7,ตรัง,Trang,หาดสำราญ,Hat Samran
8,ชุมพร,Chumphon,ปะทิว,Pathio
9,ชุมพร,Chumphon,ทุ่งตะโก,Thung Tako


In [38]:
# Group dataframe by province (CHANGWAT) and area (AMPHOE)

df_coord_gr = df_coord.groupby(['CHANGWAT_E','AMPHOE_E']).mean()
df_coord_gr.head(350)

LAT        LONG
CHANGWAT_E     AMPHOE_E                                    
Amnat Charoen  Chanuman               16.130000  104.938400
               Hua Taphan             15.688750  104.524750
               Lue Amnat              15.714857  104.693286
               Mueang Amnat Charoen   15.874000  104.630684
               Pathum Ratchawongsa    15.885429  104.899571
               Phana                  15.691250  104.870750
               Senangkhanikhom        16.053000  104.680500
Ang Thong      Chaiyo                 14.674667  100.468222
               Mueang Ang Thong       14.583000  100.449000
               Pa Mok                 14.490250  100.453250
               Pho Thong              14.670067  100.369000
               Samko                  14.598600  100.261000
               Sawaeng Ha             14.747714  100.300143
               Wiset Chai Chan        14.554867  100.323400
Bangkok        Bang Bon               13.646000  100.370000
               Bang Kapi              13.771000  100.647500
               Bang Khae              13.710500  100.397500
               Bang Khen              13.867000  100.628000
               Bang Kho Laem          13.698667  100.506000
               Bang Khun Thian        13.580000  100.414500
               Bang Na                13.672000  100.616000
               Bang Phlat             13.787000  100.493250
               Bang Rak               13.728000  100.523200
               Bang Su                13.820000  100.529000
               Bangkok Noi            13.763400  100.472800
               Bangkok Yai            13.738000  100.480500
               Bung Kum               13.808000  100.650000
               Chatuchak              13.826000  100.565000
               Chom Thong             13.690250  100.465500
               Din Daeng              13.778000  100.567000
               Don Mueang             13.925000  100.593000
               Dusit                  13.772800  100.515400
               Huai Khwang            13.772333  100.580667
               Khanna Yao             13.821000  100.677000
               Khlong Sam Wa          13.867800  100.739800
               Khlong San             13.726500  100.502500
               Khlong Toei            13.713333  100.578667
               Lak Si                 13.877000  100.572000
               Lat Krabang            13.743000  100.789167
               Lat Phrao              13.825500  100.606000
               Min Buri               13.808000  100.748500
               Nong Chok              13.851250  100.856125
               Nong Khaem             13.697000  100.354500
               Pathum Wan             13.740500  100.531750
               Phasi Charoen          13.726714  100.445571
               Phaya Thai             13.782000  100.545000
               Phra Khanong           13.692000  100.626000
               Phra Nakhon            13.756417  100.499250
               Pom Prap Sattruphai    13.751200  100.511400
               Prawet                 13.695333  100.669667
               Rat Burana             13.672500  100.500500
               Rat Thewi              13.756000  100.538750
               Sa Thon                13.714000  100.529000
               Sai Mai                13.905000  100.652000
               Samphanthawong         13.738000  100.509333
               Saphan Sung            13.761000  100.688000
               Suan Luang             13.726000  100.628000
               Taling Chan            13.767500  100.439833
               Thawi Watthana         13.770500  100.369000
               Thon Buri              13.722000  100.484800
               Thung Khu              13.632500  100.503500
               Wang Thong Lang        13.779000  100.609000
               Watthana               13.732667  100.577333
               Yannawa                13.689500  100.538500
Bueng Kan      Bueng Kan              18.318417  103.581167
               Bueng Kho

#### 3.2.2 Constituency Data

In [39]:
# Convert from json file into dataframe

zone = result_zone

df_zone = json_normalize(zone)

df_zone.head()

,areas,prefixes.area,prefixes.sub_area,province,zone
0,"[{'exterior': [], 'subinterior': [], 'area': '...",อำเภอ,ตำบล,กระบี่,1
1,"[{'exterior': [], 'subinterior': [], 'area': '...",อำเภอ,ตำบล,กระบี่,2
2,"[{'exterior': ['ถนนนครไชยศรี'], 'subinterior':...",เขต,แขวง,กรุงเทพมหานคร,1
3,"[{'exterior': [], 'subinterior': [], 'area': '...",เขต,แขวง,กรุงเทพมหานคร,2
4,"[{'exterior': [], 'subinterior': [], 'area': '...",เขต,แขวง,กรุงเทพมหานคร,3


In [40]:
# Drop unnecessary columns and rearrange column order
df_temp = df_zone.copy()
df_zone = df_temp[["province","areas"]].copy()
df_zone_2 = df_temp[["zone"]].copy()

df_zone.head()

,province,areas
0,กระบี่,"[{'exterior': [], 'subinterior': [], 'area': '..."
1,กระบี่,"[{'exterior': [], 'subinterior': [], 'area': '..."
2,กรุงเทพมหานคร,"[{'exterior': ['ถนนนครไชยศรี'], 'subinterior':..."
3,กรุงเทพมหานคร,"[{'exterior': [], 'subinterior': [], 'area': '..."
4,กรุงเทพมหานคร,"[{'exterior': [], 'subinterior': [], 'area': '..."


In [41]:
# Flatten 'areas' from json format into comma-separated format
# Translate province and area names from Thai to English

for i in range(len(df_zone)):
    temp = ''
    for j in range(len(df_zone.loc[i,"areas"])):
        try:
            temp2 = df_coord_dict[df_coord_dict['AMPHOE_T'] == df_zone.loc[i,"areas"][j]['area']].reset_index().loc[0,'AMPHOE_E']
        except:
            if(df_zone.loc[i,"areas"][j]['area']=='ทั้งจังหวัด'):
                temp2 = 'all'
            else:
                temp2 = 'Not Found'
        if j < len(df_zone.loc[i,"areas"])-1:
            temp = temp + temp2 + ','
        else:
            temp = temp + temp2 + ''
    try:
        df_zone.loc[i,"areas"] = temp.split(',').remove('Not Found')
    except:
        df_zone.loc[i,"areas"] = temp.split(',')

    df_zone.loc[i,"province"] = df_coord_dict[df_coord_dict['CHANGWAT_T'] == df_zone.loc[i,"province"]].reset_index().loc[0,'CHANGWAT_E']
    
df_zone.head()

,province,areas
0,Krabi,"[Plai Phraya, Ao Luek, Khao Phanom, Mueang Krabi]"
1,Krabi,"[Khlong Thom, Lam Thap, Ko Lanta, Khao Phanom,..."
2,Bangkok,"[Dusit, Pom Prap Sattruphai, Phra Nakhon, Samp..."
3,Bangkok,"[Bang Rak, Pathum Wan, Sa Thon]"
4,Bangkok,"[Bang Kho Laem, Yannawa]"


In [42]:
# Find average coordinate for each row

for i in range(len(df_zone)):
    df_zone.loc[i,"lat"]=['']
    df_zone.loc[i,"long"]=['']
    lat = ''
    long = ''
    if df_zone.loc[i,"areas"] is not None:
        
        for j in range(len(df_zone.loc[i,"areas"])):

            try:
                temp_lat = df_coord_gr.loc[(df_zone.loc[i,"province"],df_zone.loc[i,"areas"][j]),'LAT']
                temp_long = df_coord_gr.loc[(df_zone.loc[i,"province"],df_zone.loc[i,"areas"][j]),'LONG']

            except:

                temp_lat = df_coord_gr.loc[df_zone.loc[i,"province"],'LAT'].mean()
                temp_long = df_coord_gr.loc[df_zone.loc[i,"province"],'LONG'].mean()

            if j < len(df_zone.loc[i,"areas"])-1:
                lat = lat + str(temp_lat) + ','
                long = long + str(temp_long) + ','
            else:
                lat = lat + str(temp_lat)
                long = long + str(temp_long)

        df_zone.loc[i,"lat"] = np.mean(list(map(float, lat.split(','))))
        df_zone.loc[i,"long"] = np.mean(list(map(float, long.split(','))))
    else:
        df_zone.loc[i,"lat"] = df_coord_gr.loc[df_zone.loc[i,"province"],'LAT'].mean()
        df_zone.loc[i,"long"] = df_coord_gr.loc[df_zone.loc[i,"province"],'LONG'].mean()
    
df_zone.head(350)

,province,areas,lat,long
0,Krabi,"[Plai Phraya, Ao Luek, Khao Phanom, Mueang Krabi]",8.29504,98.86
1,Krabi,"[Khlong Thom, Lam Thap, Ko Lanta, Khao Phanom,...",7.94758,99.1607
2,Bangkok,"[Dusit, Pom Prap Sattruphai, Phra Nakhon, Samp...",13.7546,100.509
3,Bangkok,"[Bang Rak, Pathum Wan, Sa Thon]",13.7275,100.528
4,Bangkok,"[Bang Kho Laem, Yannawa]",13.6941,100.522
5,Bangkok,"[Khlong Toei, Watthana]",13.723,100.578
6,Bangkok,"[Din Daeng, Huai Khwang]",13.7752,100.574
7,Bangkok,"[Chatuchak, Phaya Thai, Rat Thewi]",13.788,100.55
8,Bangkok,"[Dusit, Bang Su]",13.7964,100.522
9,Bangkok,"[Lat Phrao, Wang Thong Lang]",13.8023,100.607


In [43]:
# Join back with zone columns
df_zone = pd.concat([df_zone, df_zone_2], axis=1, join='inner')
df_zone.head()

,province,areas,lat,long,zone
0,Krabi,"[Plai Phraya, Ao Luek, Khao Phanom, Mueang Krabi]",8.29504,98.86,1
1,Krabi,"[Khlong Thom, Lam Thap, Ko Lanta, Khao Phanom,...",7.94758,99.1607,2
2,Bangkok,"[Dusit, Pom Prap Sattruphai, Phra Nakhon, Samp...",13.7546,100.509,1
3,Bangkok,"[Bang Rak, Pathum Wan, Sa Thon]",13.7275,100.528,2
4,Bangkok,"[Bang Kho Laem, Yannawa]",13.6941,100.522,3


In [44]:
# Finalize neighborhood coordinate dataframe
neigh = df_zone[["province","zone","lat","long"]]
neigh = neigh.sort_values(['province', 'zone']).reset_index(drop=True)
neigh.head()

,province,zone,lat,long
0,Amnat Charoen,1,15.7814,104.578
1,Amnat Charoen,2,15.8949,104.817
2,Ang Thong,1,14.617,100.375
3,Bangkok,1,13.7546,100.509
4,Bangkok,2,13.7275,100.528


#### 3.2.3 Election Result

In [45]:
# Import XLSX data and convert into dataframe format

df_result = pd.read_excel('https://github.com/pornvutp/Coursera_Capstone/blob/master/election_result.xlsx?raw=true')
df_result.head()

,province,zone,party,num_votes,rank
0,กรุงเทพมหานคร,1,พลังประชารัฐ,23246,1
1,กรุงเทพมหานคร,2,พลังประชารัฐ,26909,1
2,กรุงเทพมหานคร,3,อนาคตใหม่,28444,1
3,กรุงเทพมหานคร,4,พลังประชารัฐ,27620,1
4,กรุงเทพมหานคร,5,เพื่อไทย,27897,1


In [46]:
# Eliminate 'rank' and 'num_votes' column since we focus only on the winner for each constituency
df_result = df_result[['province','zone','party']]
df_result.head()

,province,zone,party
0,กรุงเทพมหานคร,1,พลังประชารัฐ
1,กรุงเทพมหานคร,2,พลังประชารัฐ
2,กรุงเทพมหานคร,3,อนาคตใหม่
3,กรุงเทพมหานคร,4,พลังประชารัฐ
4,กรุงเทพมหานคร,5,เพื่อไทย


In [47]:
# Translate province from Thai to English

for i in range(len(df_result)):
    
    df_result.loc[i,"province"] = df_coord_dict[df_coord_dict['CHANGWAT_T'] == df_result.loc[i,"province"]].reset_index().loc[0,'CHANGWAT_E']
    
df_result.head()

,province,zone,party
0,Bangkok,1,พลังประชารัฐ
1,Bangkok,2,พลังประชารัฐ
2,Bangkok,3,อนาคตใหม่
3,Bangkok,4,พลังประชารัฐ
4,Bangkok,5,เพื่อไทย


In [48]:
df_result = df_result.sort_values(['province', 'zone']).reset_index(drop=True)
df_result.head()

,province,zone,party
0,Amnat Charoen,1,เพื่อไทย
1,Amnat Charoen,2,เพื่อไทย
2,Ang Thong,1,ภูมิใจไทย
3,Bangkok,1,พลังประชารัฐ
4,Bangkok,2,พลังประชารัฐ


In [49]:
df_final = pd.concat([neigh, df_result], axis=1, join='inner')
df_final = df_final.loc[:,~df_final.columns.duplicated()]
df_final.head()

,province,zone,lat,long,party
0,Amnat Charoen,1,15.7814,104.578,เพื่อไทย
1,Amnat Charoen,2,15.8949,104.817,เพื่อไทย
2,Ang Thong,1,14.617,100.375,ภูมิใจไทย
3,Bangkok,1,13.7546,100.509,พลังประชารัฐ
4,Bangkok,2,13.7275,100.528,พลังประชารัฐ


In [50]:
# Create new 'key' column

for i in range(len(df_final)):
    
    df_final.loc[i,"key"] = str(df_final.loc[i,"province"]) + " " + str(df_final.loc[i,"zone"])
    
df_final.head()

,province,zone,lat,long,party,key
0,Amnat Charoen,1,15.7814,104.578,เพื่อไทย,Amnat Charoen 1
1,Amnat Charoen,2,15.8949,104.817,เพื่อไทย,Amnat Charoen 2
2,Ang Thong,1,14.617,100.375,ภูมิใจไทย,Ang Thong 1
3,Bangkok,1,13.7546,100.509,พลังประชารัฐ,Bangkok 1
4,Bangkok,2,13.7275,100.528,พลังประชารัฐ,Bangkok 2


### 3.3 Exploring Dataset

In [51]:
address = 'Thailand'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of {} are {}, {}.'.format(address,latitude, longitude))

The geograpical coordinate of Thailand are 14.8971921, 100.83273.


In [52]:
# create map using latitude and longitude values
map_th = folium.Map(location=[latitude, longitude], zoom_start=6)

# add markers to map
for lat, lng, province, zone, party in zip(df_final['lat'], df_final['long'], df_final['province'], df_final['zone'], df_final['party']):
    label = '{}, {}'.format(province, zone)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_th)  
    
map_th

In [53]:
# Define function to get nearby venues for each constituency


CLIENT_ID = '1IIMSWR54DHDW1TBJCVEUPJRMUOOT1V12EAVAKGQ3VVQTVA0' # your Foursquare ID
CLIENT_SECRET = 'UWVK2RNVMJBJC2GQWGZEKHAGSFRENRU4M5YJGJ35LBRR1UXE' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)


LIMIT = 100 # limit of number of venues returned by Foursquare API


def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)


Your credentails:
CLIENT_ID: 1IIMSWR54DHDW1TBJCVEUPJRMUOOT1V12EAVAKGQ3VVQTVA0
CLIENT_SECRET:UWVK2RNVMJBJC2GQWGZEKHAGSFRENRU4M5YJGJ35LBRR1UXE


In [54]:
# Use the defined function 

th_venues = getNearbyVenues(names=df_final['key'],
                                   latitudes=df_final['lat'],
                                   longitudes=df_final['long']
                                  )

Amnat Charoen 1
Amnat Charoen 2
Ang Thong 1
Bangkok 1
Bangkok 2
Bangkok 3
Bangkok 4
Bangkok 5
Bangkok 6
Bangkok 7
Bangkok 8
Bangkok 9
Bangkok 10
Bangkok 11
Bangkok 12
Bangkok 13
Bangkok 14
Bangkok 15
Bangkok 16
Bangkok 17
Bangkok 18
Bangkok 19
Bangkok 20
Bangkok 21
Bangkok 22
Bangkok 23
Bangkok 24
Bangkok 25
Bangkok 26
Bangkok 27
Bangkok 28
Bangkok 29
Bangkok 30
Bueng Kan 1
Bueng Kan 2
Buri Ram 1
Buri Ram 2
Buri Ram 3
Buri Ram 4
Buri Ram 5
Buri Ram 6
Buri Ram 7
Buri Ram 8
Chachoengsao 1
Chachoengsao 2
Chachoengsao 3
Chachoengsao 4
Chai Nat 1
Chai Nat 2
Chaiyaphum 1
Chaiyaphum 2
Chaiyaphum 3
Chaiyaphum 4
Chaiyaphum 5
Chaiyaphum 6
Chanthaburi 1
Chanthaburi 2
Chanthaburi 3
Chiang Mai 1
Chiang Mai 2
Chiang Mai 3
Chiang Mai 4
Chiang Mai 5
Chiang Mai 6
Chiang Mai 7
Chiang Mai 8
Chiang Mai 9
Chiang Rai 1
Chiang Rai 2
Chiang Rai 3
Chiang Rai 4
Chiang Rai 5
Chiang Rai 6
Chiang Rai 7
Chon Buri 1
Chon Buri 2
Chon Buri 3
Chon Buri 4
Chon Buri 5
Chon Buri 6
Chon Buri 7
Chon Buri 8
Chumphon 1
Chumph

In [55]:
# View example of venues nearby the constituencies

print(th_venues.shape)
th_venues.head(100)

(8610, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Amnat Charoen 1,15.781375,104.577717,Fai Kid Hotel (โรงแรมฝ้ายขิด),15.860842,104.620121,Hotel
1,Amnat Charoen 1,15.781375,104.577717,แผ่นดินทองฟาร์มฟิชชิ่งปาร์ค,15.834809,104.606749,American Restaurant
2,Amnat Charoen 1,15.781375,104.577717,ก๋วยเตี๋ยวเป็ดซิตี้,15.863962,104.584794,Noodle House
3,Amnat Charoen 1,15.781375,104.577717,Shell (หอมกาแฟ),15.862201,104.599185,Café
4,Amnat Charoen 1,15.781375,104.577717,Sirin Cafe,15.853887,104.631146,Coffee Shop
5,Amnat Charoen 1,15.781375,104.577717,ส้มตำยายพัน,15.858168,104.625800,Thai Restaurant
6,Amnat Charoen 2,15.894907,104.816501,ผาชื่นวาริน,15.910299,104.796067,National Park
7,Amnat Charoen 2,15.894907,104.816501,ภูเปาะ จ.อํานาจเริญ,15.940458,104.850145,Mountain
8,Amnat Charoen 2,15.894907,104.816501,ห้วยสีโท,15.886124,104.740519,Travel & Transport
9,Amnat Charoen 2,15.894907,104.816501,วัดดอยนกยูงทอง,15.954864,104.872673,Forest


In [56]:
# Group venues by 'Neighborhood' and count number of venues

th_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Amnat Charoen 1,6,6,6,6,6,6
Amnat Charoen 2,4,4,4,4,4,4
Ang Thong 1,30,30,30,30,30,30
Bangkok 1,95,95,95,95,95,95
Bangkok 10,56,56,56,56,56,56
Bangkok 11,67,67,67,67,67,67
Bangkok 12,49,49,49,49,49,49
Bangkok 13,29,29,29,29,29,29
Bangkok 14,77,77,77,77,77,77


In [57]:
# Find number of unique categories in the data extracted
print('There are {} uniques categories.'.format(len(th_venues['Venue Category'].unique())))

There are 335 uniques categories.


### 3.4 Analyze each neighborhood

#### 3.4.1 Use One Hot Encoding to create dummie variables

In [58]:
# one hot encoding
th_onehot = pd.get_dummies(th_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
th_onehot['Neighborhood'] = th_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [th_onehot.columns[-1]] + list(th_onehot.columns[:-1])
th_onehot = th_onehot[fixed_columns]

th_onehot.head()

th_grouped = th_onehot.groupby('Neighborhood').mean().reset_index()

In [59]:
th_grouped.shape


(348, 336)

#### 3.4.2 Extract top 10 most frequent venue categories

In [60]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = th_grouped['Neighborhood']

for ind in np.arange(th_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(th_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Amnat Charoen 1,Coffee Shop,Noodle House,American Restaurant,Thai Restaurant,Hotel,Café,Zoo Exhibit,Farm,Farmers Market,Fast Food Restaurant
1,Amnat Charoen 2,Forest,National Park,Travel & Transport,Mountain,Flea Market,Farm,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop
2,Ang Thong 1,Thai Restaurant,Market,Coffee Shop,Noodle House,Convenience Store,Flea Market,Soccer Stadium,Cafeteria,Asian Restaurant,Restaurant
3,Bangkok 1,Noodle House,Café,Chinese Restaurant,Thai Restaurant,Hotel,Asian Restaurant,Dessert Shop,Convenience Store,Coffee Shop,Hostel
4,Bangkok 10,Noodle House,Thai Restaurant,Convenience Store,Coffee Shop,Restaurant,Flea Market,Asian Restaurant,Steakhouse,Bar,Café


### 3.5 Clustering Neighborhoods

In [61]:
# set number of clusters
kclusters = 5

th_grouped_clustering = th_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(th_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [ ]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

In [64]:
neighborhoods_venues_sorted.head()

,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,0,Amnat Charoen 1,Coffee Shop,Noodle House,American Restaurant,Thai Restaurant,Hotel,Café,Zoo Exhibit,Farm,Farmers Market,Fast Food Restaurant
1,0,Amnat Charoen 2,Forest,National Park,Travel & Transport,Mountain,Flea Market,Farm,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop
2,0,Ang Thong 1,Thai Restaurant,Market,Coffee Shop,Noodle House,Convenience Store,Flea Market,Soccer Stadium,Cafeteria,Asian Restaurant,Restaurant
3,0,Bangkok 1,Noodle House,Café,Chinese Restaurant,Thai Restaurant,Hotel,Asian Restaurant,Dessert Shop,Convenience Store,Coffee Shop,Hostel
4,0,Bangkok 10,Noodle House,Thai Restaurant,Convenience Store,Coffee Shop,Restaurant,Flea Market,Asian Restaurant,Steakhouse,Bar,Café


In [65]:
th_merged = df_final

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
th_merged = th_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='key')

th_merged.head() # check the last columns!

,province,zone,lat,long,party,key,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Amnat Charoen,1,15.7814,104.578,เพื่อไทย,Amnat Charoen 1,0.0,Coffee Shop,Noodle House,American Restaurant,Thai Restaurant,Hotel,Café,Zoo Exhibit,Farm,Farmers Market,Fast Food Restaurant
1,Amnat Charoen,2,15.8949,104.817,เพื่อไทย,Amnat Charoen 2,0.0,Forest,National Park,Travel & Transport,Mountain,Flea Market,Farm,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop
2,Ang Thong,1,14.617,100.375,ภูมิใจไทย,Ang Thong 1,0.0,Thai Restaurant,Market,Coffee Shop,Noodle House,Convenience Store,Flea Market,Soccer Stadium,Cafeteria,Asian Restaurant,Restaurant
3,Bangkok,1,13.7546,100.509,พลังประชารัฐ,Bangkok 1,0.0,Noodle House,Café,Chinese Restaurant,Thai Restaurant,Hotel,Asian Restaurant,Dessert Shop,Convenience Store,Coffee Shop,Hostel
4,Bangkok,2,13.7275,100.528,พลังประชารัฐ,Bangkok 2,0.0,Café,Hotel,Coffee Shop,Chinese Restaurant,Asian Restaurant,Japanese Restaurant,Thai Restaurant,Ramen Restaurant,Noodle House,French Restaurant


In [66]:
th_merged.dropna(axis=0,inplace=True)


In [68]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=6)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(th_merged['lat'], th_merged['long'], th_merged['key'], th_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster-1)],
        fill=True,
        fill_color=rainbow[int(cluster-1)],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### 3.6 Explore Each Clusters

We can see that each cluster have different 

#### 3.6.1 Cluster 1

In [70]:
th_merged.loc[th_merged['Cluster Labels'] == 0, th_merged.columns[[1] + list(range(5, th_merged.shape[1]))]]

,zone,key,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,1,Amnat Charoen 1,0.0,Coffee Shop,Noodle House,American Restaurant,Thai Restaurant,Hotel,Café,Zoo Exhibit,Farm,Farmers Market,Fast Food Restaurant
1,2,Amnat Charoen 2,0.0,Forest,National Park,Travel & Transport,Mountain,Flea Market,Farm,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop
2,1,Ang Thong 1,0.0,Thai Restaurant,Market,Coffee Shop,Noodle House,Convenience Store,Flea Market,Soccer Stadium,Cafeteria,Asian Restaurant,Restaurant
3,1,Bangkok 1,0.0,Noodle House,Café,Chinese Restaurant,Thai Restaurant,Hotel,Asian Restaurant,Dessert Shop,Convenience Store,Coffee Shop,Hostel
4,2,Bangkok 2,0.0,Café,Hotel,Coffee Shop,Chinese Restaurant,Asian Restaurant,Japanese Restaurant,Thai Restaurant,Ramen Restaurant,Noodle House,French Restaurant
5,3,Bangkok 3,0.0,Convenience Store,Coffee Shop,Chinese Restaurant,Thai Restaurant,Badminton Court,Spa,Noodle House,Pub,Food Truck,Gym / Fitness Center
6,4,Bangkok 4,0.0,Thai Restaurant,Café,Hotel,Coffee Shop,Noodle House,Japanese Restaurant,Bar,BBQ Joint,French Restaurant,Korean Restaurant
7,5,Bangkok 5,0.0,Hotel,Thai Restaurant,Som Tum Restaurant,Chinese Restaurant,Asian Restaurant,Seafood Restaurant,Coffee Shop,Dim Sum Restaurant,Noodle House,Other Nightlife
8,6,Bangkok 6,0.0,Noodle House,Asian Restaurant,Som Tum Restaurant,Restaurant,Thai Restaurant,Bank,Convenience Store,Japanese Restaurant,Coffee Shop,Café
9,7,Bangkok 7,0.0,Thai Restaurant,Convenience Store,Coffee Shop,Noodle House,BBQ Joint,Fast Food Restaurant,Chinese Restaurant,Som Tum Restaurant,Steakhouse,Clothing Store


#### 3.6.2 Cluster 2

In [71]:
th_merged.loc[th_merged['Cluster Labels'] == 1, th_merged.columns[[1] + list(range(5, th_merged.shape[1]))]]

,zone,key,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
50,2,Chaiyaphum 2,1.0,River,Farm,Flea Market,Food Stand,Food Service,Exhibit,Fair,Food Truck,Farmers Market,Fast Food Restaurant
53,5,Chaiyaphum 5,1.0,Restaurant,Bus Station,Flea Market,Fair,Farm,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop,Fish Market
86,2,Kalasin 2,1.0,Flea Market,Furniture / Home Store,Plaza,Historic Site,Coffee Shop,Fish Market,Fair,Farm,Farmers Market,Fast Food Restaurant
89,5,Kalasin 5,1.0,Restaurant,Asian Restaurant,Market,Deli / Bodega,Fair,Farm,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop
103,5,Khon Kaen 5,1.0,Scenic Lookout,Science Museum,Restaurant,National Park,Park,Mountain,Zoo Exhibit,Fish Market,Fair,Farm
114,4,Lampang 4,1.0,Restaurant,Market,Thai Restaurant,Juice Bar,Mountain,Zoo Exhibit,Fish Market,Fair,Farm,Farmers Market
127,3,Maha Sarakham 3,1.0,Restaurant,Café,Resort,Convenience Store,Flea Market,Bar,History Museum,Exhibit,Fair,Farm
139,2,Nakhon Phanom 2,1.0,Restaurant,Flea Market,BBQ Joint,Thai Restaurant,Racetrack,Airport Terminal,Food Court,Food & Drink Shop,Fair,Farm
141,4,Nakhon Phanom 4,1.0,Restaurant,Flea Market,Food Truck,Fish Market,Exhibit,Fair,Farm,Farmers Market,Fast Food Restaurant,Field
163,2,Nakhon Si Thammarat 2,1.0,Farm,Resort,Breakfast Spot,Bistro,Flea Market,Night Market,Deli / Bodega,Cruise,Field,Fish & Chips Shop


#### 3.6.3 Cluster 3

In [73]:
th_merged.loc[th_merged['Cluster Labels'] == 2, th_merged.columns[[1] + list(range(5, th_merged.shape[1]))]]

,zone,key,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
39,5,Buri Ram 5,2.0,Thai Restaurant,Hotel,Historic Site,Som Tum Restaurant,Zoo Exhibit,Fish & Chips Shop,Exhibit,Fair,Farm,Farmers Market
41,7,Buri Ram 7,2.0,Restaurant,Historic Site,Thai Restaurant,Noodle House,Cruise,Fishing Spot,Farm,Farmers Market,Fast Food Restaurant,Creperie
48,2,Chai Nat 2,2.0,Thai Restaurant,Pharmacy,Soccer Field,Asian Restaurant,Event Space,Fair,Farm,Farmers Market,Fast Food Restaurant,Field
51,3,Chaiyaphum 3,2.0,Cave,Department Store,Thai Restaurant,Zoo Exhibit,Fishing Spot,Farm,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop
93,4,Kamphaeng Phet 4,2.0,Thai Restaurant,Resort,Diner,Comfort Food Restaurant,Fruit & Vegetable Store,Fair,Farm,Farmers Market,Fast Food Restaurant,Field
100,2,Khon Kaen 2,2.0,Thai Restaurant,Convenience Store,Seafood Restaurant,Café,Coffee Shop,Fish & Chips Shop,Fair,Farm,Farmers Market,Fast Food Restaurant
107,9,Khon Kaen 9,2.0,Thai Restaurant,Restaurant,Coffee Shop,Noodle House,Food Truck,Dance Studio,Flea Market,Farmers Market,Fast Food Restaurant,Field
110,2,Krabi 2,2.0,Thai Restaurant,Hot Spring,Dim Sum Restaurant,Gourmet Shop,Waterfall,Coffee Shop,Fish Market,Fair,Farm,Farmers Market
115,1,Lamphun 1,2.0,Thai Restaurant,Zoo Exhibit,Fishing Spot,Fair,Farm,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop,Fish Market
130,1,Mukdahan 1,2.0,Thai Restaurant,Zoo Exhibit,Fishing Spot,Fair,Farm,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop,Fish Market


#### 3.6.4 Cluster 4

In [74]:
th_merged.loc[th_merged['Cluster Labels'] == 3, th_merged.columns[[1] + list(range(5, th_merged.shape[1]))]]

,zone,key,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
11,9,Bangkok 9,3.0,Coffee Shop,Café,Convenience Store,Art Museum,Soccer Field,Grocery Store,Burger Joint,Badminton Court,Seafood Restaurant,Pool
54,6,Chaiyaphum 6,3.0,Coffee Shop,Convenience Store,Food Court,Café,Fishing Spot,Fair,Farm,Farmers Market,Fast Food Restaurant,Field
62,5,Chiang Mai 5,3.0,Hot Spring,Coffee Shop,Buddhist Temple,Flea Market,Fair,Farm,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop
63,6,Chiang Mai 6,3.0,Resort,National Park,Historic Site,Coffee Shop,Fishing Spot,Fair,Farm,Farmers Market,Fast Food Restaurant,Field
68,2,Chiang Rai 2,3.0,Coffee Shop,Convenience Store,Diner,Noodle House,Bakery,Café,Lingerie Store,Restaurant,Museum,Dessert Shop
70,4,Chiang Rai 4,3.0,Coffee Shop,Food Truck,Market,Pedestrian Plaza,Thai Restaurant,Warehouse Store,Zoo Exhibit,Fish & Chips Shop,Fair,Farm
77,4,Chon Buri 4,3.0,Coffee Shop,Convenience Store,Zoo Exhibit,Flea Market,Fair,Farm,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop
84,3,Chumphon 3,3.0,Coffee Shop,Market,Thai Restaurant,Intersection,Train Station,Garden,Flea Market,Paper / Office Supplies Store,Diner,Department Store
88,4,Kalasin 4,3.0,Coffee Shop,Restaurant,Fried Chicken Joint,Buffet,Farmers Market,Food Truck,Convenience Store,Deli / Bodega,Floating Market,Fast Food Restaurant
102,4,Khon Kaen 4,3.0,Coffee Shop,Convenience Store,Campground,Fishing Spot,Fair,Farm,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop


#### 3.6.5 Cluster 5

In [76]:
th_merged.loc[th_merged['Cluster Labels'] == 4, th_merged.columns[[1] + list(range(5, th_merged.shape[1]))]]

,zone,key,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
33,1,Bueng Kan 1,4.0,Resort,Asian Restaurant,Farm,Coffee Shop,Convenience Store,Food Stand,Fishing Spot,Fair,Food Truck,Farmers Market
42,8,Buri Ram 8,4.0,Convenience Store,Farm,Market,Garden Center,Zoo Exhibit,Flea Market,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop
52,4,Chaiyaphum 4,4.0,Convenience Store,National Park,Coffee Shop,Mountain,Scenic Lookout,Farm,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop
71,5,Chiang Rai 5,4.0,Convenience Store,Rental Car Location,Bar,Outlet Store,Zoo Exhibit,Fishing Spot,Fair,Farm,Farmers Market,Fast Food Restaurant
87,3,Kalasin 3,4.0,Convenience Store,Coffee Shop,Night Market,Food Truck,Café,Flea Market,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop
101,3,Khon Kaen 3,4.0,Convenience Store,Campground,Zoo,Hotel,Pub,Exhibit,Coffee Shop,Fish Market,Farm,Farmers Market
113,3,Lampang 3,4.0,Convenience Store,Intersection,Asian Restaurant,Gift Shop,Flea Market,Coffee Shop,Soccer Field,Trail,Restaurant,Art Gallery
118,2,Loei 2,4.0,Noodle House,Garden,Flea Market,Sculpture Garden,Convenience Store,Fair,Farm,Farmers Market,Fast Food Restaurant,Field
129,5,Maha Sarakham 5,4.0,Convenience Store,Intersection,Coffee Shop,Jewelry Store,Noodle House,Farm,Deli / Bodega,Restaurant,Arepa Restaurant,Cupcake Shop
146,5,Nakhon Ratchasima 5,4.0,Convenience Store,Coffee Shop,Supermarket,Shopping Mall,Noodle House,Dance Studio,Flea Market,Farm,Farmers Market,Fast Food Restaurant
